In [3]:
pip install opencv-contrib-python --user


  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np

#Video Uploading

cap = cv2.VideoCapture('video2.mp4')

min_Width=20
min_Height=20

count_line_pos = 400

#Intialize Substructor

algo = cv2.createBackgroundSubtractorMOG2(varThreshold = 600)

def Center(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx=x+x1
    cy=y+y1
    return cx,cy
    
detect = []

offset=6
counter=0

while True:
    ret,frame1=cap.read()
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    
    # Applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada,cv2.MORPH_CLOSE,kernel)
    countershape,h=cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    
    cv2.line(frame1,(300,count_line_pos),(900,count_line_pos),(255,127,0),3)
    
    
    for (i,c) in enumerate(countershape):
        (x,y,w,h)=cv2.boundingRect(c)
        validate_count = (w >= min_Width) and (h>= min_Height)
        if not validate_count:
            continue
            
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
        
        centre=Center(x,y,w,h)
        detect.append(centre)
        cv2.circle(frame1,centre,4,(0,0,255),-1)
        
        for (x,y) in detect:
            if y<(count_line_pos+offset) and y>(count_line_pos-offset):
                counter+=1
            cv2.line(frame1,(300,count_line_pos),(900,count_line_pos),(0,127,255),5)
            detect.remove((x,y))
            print("Vechicle Counter:"+str(counter))
            
    cv2.putText(frame1,"Vechilce counter :"+str(counter),(450,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3)
    
    
#     cv2.imshow("video Original",dilatada)
    cv2.imshow("video Original",frame1)
    
    if cv2.waitKey(1) == 13:
        break
        
cv2.destroyAllWindows()
cap.release()

Vechicle Counter:0
Vechicle Counter:0
Vechicle Counter:0
Vechicle Counter:0
Vechicle Counter:0
Vechicle Counter:0
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:1
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Counter:2
Vechicle Cou

In [1]:
import cv2
import numpy as np

from vehicle_detector import VehicleDetector
# Load Veichle Detector
vd = VehicleDetector()

cap = cv2.VideoCapture('video.mp4')
# algo = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret,frame1=cap.read()
    

    vehicles_folder_count = 0

    vehicle_boxes = vd.detect_vehicles(frame1)
    vehicle_count = len(vehicle_boxes)


        # Update total count
    vehicles_folder_count += vehicle_count

    for box in vehicle_boxes:
        x, y, w, h = box

        cv2.rectangle(frame1, (x, y), (x + w, y + h), (25, 0, 180), 3)

        cv2.putText(frame1, "Vehicles: " + str(vehicle_count), (20, 50), 1, 2, (0, 0, 0), 3)
#         a = vehicle_count

    cv2.imshow("video Original",frame1) 
#     print(a)
    if cv2.waitKey(10) == 13:
        break
#     print('The Lane 1 has vehicle count of :',a)
cv2.destroyAllWindows()    

In [5]:
import cv2
import numpy as np

class VehicleDetector:

    def __init__(self):
        # Load Network
        net = cv2.dnn.readNet("best.pt")
        self.model = cv2.dnn_DetectionModel(net)
        self.model.setInputParams(size=(832, 832), scale=1 / 255)


        # Allow classes containing Vehicles only
        self.classes_allowed = [2, 3, 5, 6, 7]


    def detect_vehicles(self, img):
        # Detect Objects
        vehicles_boxes = []
        class_ids, scores, boxes = self.model.detect(img, nmsThreshold=0.4)
        for class_id, score, box in zip(class_ids, scores, boxes):
            if score < 0.5:
                # Skip detection with low confidence
                continue

            if class_id in self.classes_allowed:
                vehicles_boxes.append(box)

        return vehicles_boxes
    
